In [6]:
import torch
from unsloth import FastLanguageModel
import src.config as config
from src.train.lora_train import train_model
from src.data.prompt import mbpp
from src.data.prompt import openthoughts
from src.data.prompt import pythoncodes

from huggingface_hub import snapshot_download

In [2]:
snapshot_download(
    repo_id=config.MODEL_NAME,
    local_dir=config.MODEL_PATH,
    local_dir_use_symlinks=False,
    resume_download=True
)

'/workspace/models/qwen3-0.6b'

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.SFT_MODEL_PATH, # Путь, куда сохранял (checkpoints или final)
    max_seq_length = config.MAX_TOKENS,
    dtype = None,
    load_in_4bit = True,
)

In [ ]:
model, tokenizer = train_model(pythoncodes.get_prepared_dataset)
FastLanguageModel.for_inference(model=model)

13:23:41 | INFO | Загрузка модели ./models/qwen3-0.6b...
==((====))==  Unsloth 2026.1.2: Fast Qwen3 patching. Transformers: 4.57.1. vLLM: 0.11.2.
   \\   /|    NVIDIA GeForce RTX 5070 Laptop GPU. Num GPUs = 1. Max memory: 7.96 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


13:24:11 | INFO | Составляем датасет...


/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


13:24:39 | INFO | Начинаем обучение...


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,200 | Num Epochs = 9,223,372,036,854,775,807 | Total steps = 300
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 20,185,088 of 616,235,008 (3.28% trained)
Too many dataloader workers: 4 (max is dataset.num_shards=2). Stopping 2 dataloader workers.
[datasets.iterable_dataset|WARNING]Too many dataloader workers: 4 (max is dataset.num_shards=2). Stopping 2 dataloader workers.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,3.016800
50,1.712300
75,1.199500
100,1.063800
125,1.023800
150,0.958700
175,0.965700
200,0.918100
225,0.884400
250,0.889300


13:45:37 | INFO | Сохраняем объединенную модель в ./models/qwen3-0.6b-sft...
13:45:37 | INFO | Обучение завершено!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024, padding_idx=151643)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [9]:
def quick_sanity_check(model, tokenizer, prompt):
    """
    Быстрая проверка модели на одном примере.
    Использует настройки из src.config, но адаптирует их для проверки.
    """
    print(f"🧐 Проверяем промпт: {prompt[:50]}...")

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    gen_kwargs = config.SAMPLING_SETTINGS.copy()

    gen_kwargs.update({
        "temperature": 0.6,
        "do_sample": True,
        "max_new_tokens": config.MAX_TOKENS,
        "num_return_sequences": 1,
    })

    if "n" in gen_kwargs:
        gen_kwargs["num_return_sequences"] = gen_kwargs.pop("n")

    for key in ["logprobs", "stop", "ignore_eos", "detokenize"]:
        gen_kwargs.pop(key, None)

    if "max_tokens" in gen_kwargs:
        gen_kwargs["max_new_tokens"] = gen_kwargs.pop("max_tokens")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            pad_token_id=tokenizer.eos_token_id,
            **gen_kwargs
        )

    generated_ids = outputs[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(generated_ids, skip_special_tokens=True)

In [10]:
mbpp_data = mbpp.get_dataset()["test"]
test_sample = mbpp_data[0]

prompt_data = mbpp.build_prompt(test_sample, tokenizer, train=False)
test_prompt = prompt_data["text"]

print("="*40)
print("ВХОДНОЙ ПРОМПТ:")
print(test_prompt)
print("="*40)

result = quick_sanity_check(model, tokenizer, test_prompt)

print("\n" + "="*40)
print("🤖 ОТВЕТ МОДЕЛИ:")
print("="*40)
print(result)

ВХОДНОЙ ПРОМПТ:
<|im_start|>system
You are an expert Python coding assistant. Given a problem description and function signature, implement the function body so that it passes all tests.<|im_end|>
<|im_start|>user
Problem:
Write a python function to remove first and last occurrence of a given character from the string.

Use the following function signature:
def remove_Occ(s,ch):

Write the full Python function implementation. Do NOT change the function name or arguments. Return only Python code.<|im_end|>
<|im_start|>assistant

🧐 Проверяем промпт: <|im_start|>system
You are an expert Python coding...

🤖 ОТВЕТ МОДЕЛИ:
<think>

</think>

```python
def remove_Occ(s, ch):
    # Iteration
    i = 0
    while i < len(s):
        if s[i] == ch:
            s = s[:i] + s[i+1:]
        i += 1
    return s
```
